In [3]:
import itertools
import math
import os
import pickle

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import yaml

# from annoy import AnnoyIndex
from tqdm.notebook import tqdm

In [4]:
sys.path.append("../yolov5")
from models.common import AutoShape, DetectMultiBackend

In [5]:
model = AutoShape(DetectMultiBackend("../models/yolov5m.pt"))

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


In [13]:
root = "../datasets/custom/images/"
images = [root + x for x in os.listdir(root)]
images

['../datasets/custom/images/00000001_020.jpeg',
 '../datasets/custom/images/00000001_008.jpeg',
 '../datasets/custom/images/00000001_012.jpeg',
 '../datasets/custom/images/00000001_000.jpeg',
 '../datasets/custom/images/00000001_005.jpeg',
 '../datasets/custom/images/00000001_024.jpeg',
 '../datasets/custom/images/00000001_017.jpeg',
 '../datasets/custom/images/00000001_011.jpeg']

In [7]:
import jupyter_innotater as ji

In [8]:
repeats = 4

breeds = ['cat_{}'.format(x) for x in range(repeats)]
targets_exclude = np.zeros((len(images), 1), dtype='int')
targets_breed = np.zeros((len(images), len(breeds)), dtype='int')
targets_bboxes = np.zeros((len(images), repeats, 4), dtype='int') # (x,y,w,h) for each animal


In [9]:
# Размечаем картинки
ji.Innotater(
    [
        ji.ImageInnotation(images), # Display the image itself
        ji.TextInnotation(images, multiline=False) # Display the image filename
    ],
    [
        ji.BinaryClassInnotation(targets_exclude, name='Exclude'), # Checkbox
        ji.RepeatInnotation(
            (ji.BoundingBoxInnotation, targets_bboxes), # Individual animal bounding box
            (ji.MultiClassInnotation, targets_breed,
                {'name':'Breed', 'classes':breeds, 'dropdown':True}), # Per-animal breed dropdown
            max_repeats=repeats, min_repeats=1
        )
    ]
)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [10]:
def xywh2yolo(xywh, size):
    x, y, w, h = xywh
    x += w / 2
    y += h / 2
    x = x / size[1]
    w = w / size[1]
    y = y / size[0]
    h = h / size[0]
    return x, y, w, h

In [14]:
# Сохраняем датасет в формате для Yolo
result = []
cache = root.replace('/images/', '/labels.cache')
if os.path.exists(cache):
    os.unlink(cache)
    
for im, bboxs in zip(images, targets_bboxes):
    annotation = '.'.join(im.replace('/images/', '/labels/').split('.')[:-1]) + '.txt'
    image = cv2.imread(im, cv2.IMREAD_COLOR)
    with open(annotation, 'wt') as f:
        for bbox in bboxs:
            if bbox.sum() == 0:
                continue
            xywh = xywh2yolo(bbox.astype(np.float32), image.shape)
            xywh = ' '.join(["{:.6f}".format(x) for x in xywh])
            f.write('{} {}\n'.format(15, xywh))

In [77]:
config = {
    'path': '../datasets/custom/',
    'train': 'images',
    'val': 'images',
    # 'test': None,
    'nc': 80,
    # 'nc': 1,
    # 'names': ['custom_object']
    'names': ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush'] 
}
# config['names'] = str(config['names'])
with open('./config.yaml', 'w') as f:
    yaml.dump(config, f)

In [78]:
!/home/asciishell/projects/hse-fsod/venv/bin/python \
    yolov5/train.py \
    --img 640 \
    --batch 16 \
    --epochs 3 \
    --data config.yaml \
    --weights models/yolov5m.pt \
    --cache 
    # --freeze 10

train: weights=models/yolov5m.pt, cfg=, data=config.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone git@github.com:AsciiShell/yolov5` to update.
YOLOv5 🚀 v6.1-119-gaa542ce torch 1.11.0+cu102 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.

In [79]:
model2 = AutoShape(DetectMultiBackend("./yolov5/runs/train/exp2/weights/best.pt"))

Fusing layers... 
Model summary: 290 layers, 21172173 parameters, 0 gradients, 49.0 GFLOPs
Adding AutoShape... 


In [80]:
model(images[0]).pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,35.687256,13.568044,493.835266,371.385132,0.803403,15,cat
1,1.056463,13.806772,139.548264,370.451660,0.365682,56,chair


In [81]:
model2(images[0]).pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,35.570694,9.221554,491.663361,374.221100,0.707715,15,cat
1,0.000000,9.815825,140.598206,374.720581,0.427241,56,chair
